# Validate Cell Density

Reproduce part of Table 3 of Aika et al 1994

<img src="Aika_etal_1994_Table3.png">

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bluepy.v2 import Cell, Circuit
from voxcell.nexus.voxelbrain import Atlas

In [ ]:
# Load circuit & atlas
circuit = Circuit('/gpfs/bbp.cscs.ch/project/proj42/circuits/O1/20180907/CircuitConfig')

atlas = Atlas.open('/gpfs/bbp.cscs.ch/project/proj42/circuits/O1/20180907/atlas/')
brain_regions = atlas.load_data('brain_regions')
rmap = atlas.load_region_map()

In [ ]:
def calc_density(mtype, region):
    region_ids = rmap.find(region, attr='acronym', with_descendants=True)
    volume = brain_regions.volume(region_ids)
    cell_count = circuit.cells.count({Cell.REGION: region, Cell.MTYPE: mtype})
    return 1e6 * cell_count / volume  # 1 / um^3 -> 10^3 / mm^3

In [ ]:
model = pd.Series({
    'CA1': calc_density(mtype='@.*', region='@mc2.*'),
    'SLM-SR': calc_density(mtype='@(SLM|SR).*', region='@mc2;(SLM|SR)'),
    'SP': calc_density(mtype='@SP.*', region='mc2;SP'),
    'PC in SP': calc_density(mtype='SP_PC', region='mc2;SP'),
    'SO': calc_density(mtype='@SO.*', region='mc2;SO'),
})

In [ ]:
exp = pd.Series([
      35.2, 1.9, 272.4, 264, 11.3
  ], index=['CA1', 'SLM-SR', 'SP', 'PC in SP', 'SO']
)
exp_sem = pd.Series([
      0.5, 0.3, 14.3, 14.6, 0.9
  ], index=['CA1', 'SLM-SR', 'SP', 'PC in SP', 'SO']
)  

In [ ]:
means = pd.DataFrame({
    'Model': model,
    'Experimental': exp,
})
errors = pd.DataFrame({
    'Experimental': exp_sem,
})
means.plot.bar(yerr=errors, rot=45);